In [23]:
from esovalue.eso import make_option_tree
from collections import defaultdict
from math import tan, pi
from networkx.convert import from_dict_of_lists
import hvplot.networkx as hvnx
import holoviews as hv
from mpmath import mpf
from tests.test_eso import black_scholes
from collections import OrderedDict

parameters = {'strike_price': mpf('111.07999999999999999999999999999999999999999999999995'), 'stock_price': mpf('102.42000000000000000000000000000000000000000000000005'), 'volatility': mpf('0.10000000000000000000000000000000000000000000000000007'), 'risk_free_rate': mpf('0.0'), 'dividend_rate': mpf('0.0'), 'exit_rate': mpf('0.0'), 'vesting_years': mpf('0.0'), 'expiration_years': mpf('0.57700000000000000000000000000000000000000000000000024'), 'm': None}

def _get_node_id(n):
    return str(id(n))


def _format_if_float(v, precision=3):
    return f"{float(v):.{precision}f}" if isinstance(v, mpf) else str(v)


def draw_option_tree(iterations: int):
    final_parameters = dict(iterations=iterations, **parameters)
    root = make_option_tree(**final_parameters)
    root_id = _get_node_id(root)
    node_positions = {root_id: (0, 0)}
    node_colors = {}
    node_labels = {}
    adjacencies = defaultdict(set)
    unexplored = [(root, 0)]
    while unexplored:
        node, depth = unexplored.pop()
        node_id = _get_node_id(node)
        node_colors[node_id] = "lightcoral" if node.option_value == 0 else "palegreen"
        print(node_id, node.option_value, node_colors[node_id])
        expiration_years = final_parameters["expiration_years"]
        dt = expiration_years / iterations
        if depth < iterations:
            bsm = black_scholes(node.stock_value, final_parameters["strike_price"], final_parameters["risk_free_rate"],
                                expiration_years - depth*dt, final_parameters["volatility"])
        else:
            bsm = "?"
        node_labels[node_id] = f"stock = {_format_if_float(node.stock_value, 2)}\noption = {_format_if_float(node.option_value, 2)}\nbsm = {_format_if_float(bsm, 2)}\n" \
                               f"bsm diff. = {_format_if_float(node.option_value - bsm, 2) if isinstance(bsm, mpf) else '?'}"
        node_x, node_y = node_positions[node_id]
        offset = tan(15/360*2*pi)
        for child, o in ((node.up, offset), (node.middle, 0), (node.down, -offset)):
            if child:
                child_id = _get_node_id(child)
                adjacencies[node_id].add(child_id)
                node_positions[child_id] = (node_x + 1, node_y + o)
                unexplored.append((child, depth+1))
            else:
                adjacencies[node_id] = set()
    adjacencies = OrderedDict(adjacencies)
    g = from_dict_of_lists(adjacencies)
    return hvnx.draw_networkx(g, node_positions, node_shape="s",
                              node_size=6000, font_size="7pt", labels=node_labels, width=900, height=600, with_labels=True,
                              node_color={"id": lambda n: node_colors[n]}) * hv.Text(0, -0.2, "\n".join([f"{p} = {_format_if_float(v)}"
                                                                                       for p, v in sorted(final_parameters.items())])).opts(hv.opts.Text(text_align='left', text_baseline="top", text_font_size="10pt"))

draw_option_tree(3)

139940880465728 0.49107473977059859616235106359639608561956310954303 palegreen
139940880449936 0.0 lightcoral
139940885289408 0.0 lightcoral
139940885292720 0.0 lightcoral
139940885291712 0.0 lightcoral
139940885284608 0.0 lightcoral
139940885284464 0.0 lightcoral
139940885291712 0.0 lightcoral
139940885284608 0.0 lightcoral
139940885291952 0.0 lightcoral
139940885291328 0.0 lightcoral
139940885284608 0.0 lightcoral
139940885291952 0.0 lightcoral
139940885291856 0.0 lightcoral
139940885288976 0.20936140806268860146010916033784317590391423180713 palegreen
139940885284464 0.0 lightcoral
139940885291712 0.0 lightcoral
139940885284608 0.0 lightcoral
139940885291952 0.0 lightcoral
139940885291328 0.0 lightcoral
139940885284608 0.0 lightcoral
139940885291952 0.0 lightcoral
139940885291856 0.0 lightcoral
139940885287248 1.3057616112984801757937954844994665579339754980844 palegreen
139940885291952 0.0 lightcoral
139940885291856 0.0 lightcoral
139940885284320 8.143876186724322291867635105470886

ValueError: failed to validate Scatter(id='6127', ...).fill_color: expected an element of either String, Dict(Enum('expr', 'field', 'value', 'transform'), Either(String, Instance(Transform), Instance(Expression), Nullable(Color))) or Nullable(Color), got {'id': <function draw_option_tree.<locals>.<lambda> at 0x7f4686d9a3b0>}

:Overlay
   .Graph.I  :Graph   [start,end]
   .Labels.I :Labels   [x,y]   (text)
   .Text.I   :Text   [x,y]